In [ ]:
# Lectura de dataframe
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
file_path = r""
df_m = pd.read_excel(file_path, sheet_name="02_GRAN", skiprows=1, usecols="E:AM")
df_t = pd.read_excel(file_path, sheet_name="GRAT")
df_g = pd.read_excel(file_path, sheet_name="GRAG")
df_b = pd.read_excel(file_path, sheet_name="01_GRAB")
df_d = pd.read_excel(file_path, sheet_name="09_DIQU")
df_l = pd.read_excel(file_path, sheet_name="LOCA")
df_tr = pd.read_excel(file_path, sheet_name="TREG")
df_id = pd.read_excel(file_path, sheet_name="IDEN")

c:\Program Files\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Program Files\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Program Files\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Program Files\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# Iterar
for i in [46]:
    #Filtrar base de datos por dique
    dique = df_d['CODIGO'][i]
    df_x2 = df_l[df_l['LOCA_NAME']==dique].reset_index(drop=True)

    #Filtrar base de datos por calicata
    df_x3 = df_g[df_g['LOCA_ID'].isin(df_x2['LOCA_ID'])].reset_index(drop=True)
    df_x3 = df_x3[df_x3['SAMP_TYPE']!='G'].reset_index(drop=True)  
    #Crear figura
    fig = plt.figure(figsize=(12.5,10), dpi = 480)

    #Agregar las curvas en la figura
    for j in range(len(df_x3)):
        g_id = df_x3['SAMP_ID'][j]
        loca =  df_x3['LOCA_ID'][j]        
        sucs = df_x3['GRAG_SUCS'][j]
        finos = round(df_x3['GRAG_FINE'][j],2)
        ip = df_x3['GRAG_PI'][j]   

        #Obtener la granulometria
        gran = df_t[df_t['SAMP_ID']==g_id].reset_index(drop=True)

        #Eliminar valores mayores a 100
        if 100 in gran["GRAT_PERP"].values:
            # Buscar la última fila donde GRAT_PERP es 100
            last_100_index = gran[gran["GRAT_PERP"] == 100].index[-1]
            
            # Seleccionar los datos desde esa fila hasta el final
            x = gran["GRAT_SIZE"].iloc[last_100_index:]
            y = gran["GRAT_PERP"].iloc[last_100_index:]
        else:
            # Si no hay 100, usar todos los datos
            x = gran["GRAT_SIZE"]
            y = gran["GRAT_PERP"]

        if loca in df_tr['LOCA_ID'].values:
            phi = df_tr.loc[df_tr['LOCA_ID'] == loca, 'TREG_PHI'].values[0]
            c = df_tr.loc[df_tr['LOCA_ID'] == loca, 'TREG_COH'].values[0]
            triaxial = f'({phi}° - {c} kPa)'
        else:
            triaxial = "NR"

        if loca in df_id['LOCA_ID'].values:
            dens = round(df_id.loc[df_id['LOCA_ID'] == loca, 'IDEN_IDEN'].values[0],2)
            densidad = f'({dens} kg/m3)'
        else:
            densidad = 'NR'
        
        if ip=="NP":
            indice = 'NP'
        else:
            indice = f'({ip}%)'
        fines = f'({finos}%)'

        label_text = (
            f"{g_id} | {sucs} | Finos: {fines} | IP: {indice} | "  # Primera línea
            f"Triaxial: {triaxial} | Densidad: {densidad}"        # Segunda línea
        )
                
        plt.plot(x, y, ls='-', lw=4, label=label_text,marker="o",ms=6, mfc="w")                       

    #Editar grafica
    ax = plt.gca()  # Obtener el objeto de los ejes actuales
    #------------------------------------------------------
    # Configurar títulos y etiquetas generales
    plt.xlabel("Tamaño de partícula [mm]", fontsize=20)
    plt.ylabel("Porcentaje que pasa [%]", fontsize=20)
    #plt.title('Arenas en Diques proyectados Tramo 3 & Tramo 4', pad=180, fontsize=24, fontweight='bold')
    #------------------------------------------------------
    # Mostrar leyenda y cuadrícula
    plt.legend(fontsize=14, bbox_to_anchor=(1.05, 0.6), loc="center left", ncol=1)  # Muestra la leyenda
    plt.grid(True, which="major", axis="y", linestyle="--", linewidth=0.5)
    plt.grid(True, which="minor", axis="x", linestyle="--", linewidth=0.5)
    plt.tick_params("both",labelsize=16)
    #------------------------------------------------------
    # Escala logarítmica para X
    plt.xscale("log")
    #Establecer limtites
    x_inf, x_sup = 0.01, 76.2
    ax.set_xlim(x_inf, x_sup)
    # Crear un eje X superior
    ax2 = ax.twiny()
    ax2.set_xlim(ax.get_xlim())  
    ax2.set_xscale("log")

    # Configurar los valores del eje superior
    valores_x = [0.075, 0.425,2.0,4.75,19.05,76.2]  
    etiquetas_x = ["N°200", "N°40","N°10","N°4",'3/4"','3"'] 
    ax2.set_xticks(valores_x)
    ax2.set_xticklabels(etiquetas_x, fontsize=16)

    # Posicionar el eje superior correctamente
    ax2.xaxis.set_ticks_position("top")  
    ax2.xaxis.set_label_position("top")  
    #------------------------------------------------------
    # Cambiar el borde de la gráfica
    ax.spines['top'].set_color('black')      # Borde superior en negro
    ax.spines['top'].set_linewidth(2)       # Grosor del borde superior
    ax.spines['bottom'].set_color('black')   # Borde inferior en negro
    ax.spines['bottom'].set_linewidth(2)    # Grosor del borde inferior
    ax.spines['left'].set_color('black')     # Borde izquierdo en negro
    ax.spines['left'].set_linewidth(2)      # Grosor del borde izquierdo
    ax.spines['right'].set_color('black')    # Borde derecho en negro
    ax.spines['right'].set_linewidth(2)     # Grosor del borde derecho
    #------------------------------------------------------
    #Ejes verticales notables
    for x in valores_x:
        ax.axvline(x, color="#2F4F4F", linestyle='-', linewidth=1)

    #------------------------------------------------------
    #Datos la tabla superior
    tabla_datos1 = [["Finos", "Arena", "Grava"]]

    #Calculo de espaciamientos - Tabla 1
    t1_width = np.log(76.2/x_inf)
    t1_graph = np.log(x_sup/x_inf)
    t1_fines = np.log(0.075/x_inf)/t1_width
    t1_sands = np.log(4.75/0.075)/t1_width
    t1_gravels = np.log(76.2/4.75)/t1_width

    #Crear la tabla
    tabla1 = ax.table(cellText=tabla_datos1, 
                    colWidths=[t1_fines,t1_sands,t1_gravels] * len(tabla_datos1[0]), 
                    cellLoc='center', 
                    loc='top', 
                    bbox=[0, 1.20, t1_width/t1_graph, 0.07])  # Ajustar la posición

    #------------------------------------------------------
    #Datos la tabla inferior
    tabla_datos2 = [["Arcilla y/o Limo", "Fina", "Media", "Gruesa", "Fina", "Gruesa"]]

    # Calculo de espaciamientos - Tabla 2
    t2_fines = np.log(0.075/x_inf)/t1_width
    t2_sands1 = np.log(0.425/0.075)/t1_width
    t2_sands2 = np.log(2/0.425)/t1_width
    t2_sands3 = np.log(4.75/2)/t1_width
    t2_gravels1 = np.log(19.05/4.75)/t1_width
    t2_gravels2 = np.log(76.2/19.05)/t1_width

    #Crear la tabla
    tabla2 = ax.table(cellText=tabla_datos2, 
                    colWidths=[t2_fines,t2_sands1,t2_sands2,t2_sands3,t2_gravels1,t2_gravels2] * len(tabla_datos2[0]), 
                    cellLoc='center', 
                    loc='top', 
                    bbox=[0, 1.10, t1_width/t1_graph, 0.07])  # Ajustar la posición

    #------------------------------------------------------           
    # Aplicar estilo a la tabla


    for key, cell in tabla1.get_celld().items():
        cell.set_linewidth(1.5)  # Grosor de los bordes
        cell.set_fontsize(16)
        cell.set_text_props(fontweight='bold', color='white')  # Aplicar negrita correctamente
        cell.set_facecolor("#2F4F4F")
        cell.set_edgecolor("white")  # Bordes blancos
        
    for key, cell in tabla2.get_celld().items():
        cell.set_linewidth(1.5)  # Grosor de los bordes
        cell.set_fontsize(16)
        cell.set_facecolor("#F5F5F5")
    #------------------------------------------------------        
    fig.savefig(rf"Imagenes\{dique}.png", format="png", bbox_inches='tight')



In [ ]:
from pylatex import Document, Package, NoEscape
from pylatex.utils import bold

def gen_pdf(Datos):

    geometry_options = {"right":"1cm", "left":"1cm", "top":"1cm", "bottom":"1cm", "landscape":True, "a4paper":True}
    doc = Document("reporte", geometry_options = geometry_options)
    doc.packages.append(Package("multirow"))
    doc.packages.append(Package("array"))
    doc.packages.append(Package("colortbl"))
    doc.packages.append(Package("xcolor", options="table"))
    doc.packages.append(Package("booktabs"))
    doc.packages.append(Package("graphicx"))
    doc.packages.append(Package("helvet"))
    doc.preamble.append(NoEscape(r"\renewcommand{\familydefault}{\sfdefault}"))
    for i in [15,16,17,19,22,24,25,26]:
        dique = df_d['CODIGO'][i]
        longitud = f"{df_d['Longitud'][i]:.2f}"
        i_formateado = f"{i+1:02d}"
        imagen_path = "Imagenes/" + dique + ".png"
        strr = r"""
            \date{}
            \pagestyle{empty}
            \thispagestyle{empty}
            \setlength{\tabcolsep}{0pt}
            \setlength{\extrarowheight}{0pt}
            \begin{table}[htbp]
            \centering
            \begin{tabular}{|m{0.15\textwidth}|m{0.20\textwidth}|m{0.20\textwidth}|m{0.20\textwidth}|m{0.10\textwidth}|m{0.10\textwidth}|m{0.05\textwidth}|}
                \hline

                \multirow{3}{*}{
                    \begin{minipage}[c][2cm][c]{0.15\textwidth}
                        \centering
                        \includegraphics[height=1.1cm,keepaspectratio]{logo.png}
                    \end{minipage}
                } &
        
                \multicolumn{3}{c|}{
                    \multirow{2}{*}{\parbox[c][1cm][c]{0.60\textwidth}{\centering \bfseries \fontsize{13}{14}\selectfont """ + Datos.title + r"""}}
                } &

                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering Elaborado:}&
                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering """ + Datos.elab + r"""} &
                \multirow{3}{*}{\parbox[c][2cm][c]{0.05\textwidth}{\centering\bfseries \fontsize{16}{14} \selectfont """ + i_formateado + r"""}} \\

                \cline{5-6}

                & \multicolumn{3}{c|}{}
                \parbox[c][0.5cm][c]{0.20\textwidth}{}&
                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering Aprobado:}&
                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering """ + Datos.aprob + r"""} &\\
                
                \cline{2-6}

                &\parbox[c][0.5cm][c]{0.20\textwidth}{\centering Proyecto:} & 
                \parbox[c][0.5cm][c]{0.20\textwidth}{\centering Subproyecto:}&
                \parbox[c][0.5cm][c]{0.20\textwidth}{\centering Estructura:}&
                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering Revisión:}&
                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering """ + Datos.rev + r"""} &\\

                \cline{2-6}

                &\parbox[c][0.5cm][c]{0.20\textwidth}{\centering """ + Datos.pro + r"""} & 
                \parbox[c][0.5cm][c]{0.20\textwidth}{\centering """ + Datos.sub + r"""}&
                \parbox[c][0.5cm][c]{0.20\textwidth}{\centering """ + dique + " [" + longitud + " m]" + r"""}&
                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering Fecha:}&
                \parbox[c][0.5cm][c]{0.10\textwidth}{\centering """ + Datos.fec + r"""} &\\

                \hline

                \multicolumn{7}{|c|}{
                        \begin{minipage}[c][16.5cm][c]{\textwidth}
                            \centering
                            \includegraphics[height=14cm]{""" + imagen_path + r"""}
                        \end{minipage}
                } \\

                \hline
            \end{tabular}
            \end{table}
            """

        doc.append(NoEscape(strr))

    doc.generate_pdf(compiler="pdflatex", clean=True)

class Datos:
    pro = "Proyecto xxx"
    sub = "Subproyecto xxx"
    est = "Estructura xxx"
    elab = "J.S."
    ngrap = "1.1"
    aprob = "H.A."
    title = "Resumen de ensayos - Analisis granulometrico"
    fec = "24/03/2025"
    rev = "0A"

gen_pdf(Datos)